<a href="https://colab.research.google.com/github/hridog00/03MAIR---Algoritmos-de-Optimizacion---2019/blob/master/Helena_Ridocci_AG_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nombre: Helena Ridocci 

Link Colab: https://colab.research.google.com/drive/1ICXuM51Ov3LvDoRN96oFCx3PrVO3znCM

Link GitHub: https://github.com/hridog00/03MAIR---Algoritmos-de-Optimizacion---2019/blob/master/Helena_Ridocci_AG_2.ipynb

Actividad Guiada 2



Porgramacion dinamica

In [0]:
tarifas = [
[0,5,4,3,999,999,999],
[999,0,999,2,3,999,11],
[999,999,0,1,999,4,10],
[999,999,999,0,5,6,9],
[999,999,999,999,0,999,4],
[999,999,999,999,999,0,3],
[999,999,999,999,999,999,0]
]

In [0]:
def get_precios(tarifas):
    N = len(tarifas[0])

    precios =  [[9999]*N for i in [9999]*N]
    ruta = [[""]*N for i in [""]*N]

    for i in range(N-1):
        for j in range(i+1, N):
            min_ =  tarifas[i][j]
            ruta[i][j] = i

            for k in range(i,j):
                if precios[i][k] + tarifas[k][j] < min_:
                    min_ = min(min_, precios[i][k] + tarifas[k][j])
                    ruta[i][j] = k
                precios[i][j] =  min_
    return precios, ruta

In [0]:
def calcular_ruta(ruta, desde, hasta):
    if desde == hasta:
        return desde
    else:
        return str(calcular_ruta(ruta, desde, ruta[desde][hasta])) + ',' + str(ruta[desde][hasta])

In [7]:
precios, ruta = get_precios(tarifas)

print("Precios")
for i in range(len(tarifas)):
    print(precios[i])

print()
print("Ruta")
for i in range(len(tarifas)):
    print(ruta[i])

Precios
[9999, 5, 4, 3, 8, 8, 11]
[9999, 9999, 999, 2, 3, 8, 7]
[9999, 9999, 9999, 1, 6, 4, 7]
[9999, 9999, 9999, 9999, 5, 6, 9]
[9999, 9999, 9999, 9999, 9999, 999, 4]
[9999, 9999, 9999, 9999, 9999, 9999, 3]
[9999, 9999, 9999, 9999, 9999, 9999, 9999]

Ruta
['', 0, 0, 0, 1, 2, 5]
['', '', 1, 1, 1, 3, 4]
['', '', '', 2, 3, 2, 5]
['', '', '', '', 3, 3, 3]
['', '', '', '', '', 4, 4]
['', '', '', '', '', '', 5]
['', '', '', '', '', '', '']


In [8]:
calcular_ruta(ruta, 0, 6)

'0,0,2,5'

Ramificacion y poda

In [0]:
costes  =[[11,12,18,40],
          [14,15,13,22],
          [11,17,19,23],
          [17,14,20,28]]

In [0]:
def valor(s, costes):
    valor_ = 0
    for i in range(len(s)):
        valor_ += costes[s[i]][i]
    return valor_

valor((0,1,2,3), costes)

73

In [0]:
import itertools
def fuerza_bruta(costes):
    mejor_valor = 10e10
    mejor_solucion = ()

    for s in list(itertools.permutations(range(len(costes)))):
        valor_tmp =  valor(s, costes)
        if valor_tmp < mejor_valor:
            mejor_valor = valor_tmp
            mejor_solucion = s
    print("La mejor solucion es :", mejor_solucion, "con valor :", mejor_valor)

fuerza_bruta(costes)

La mejor solucion es : (0, 3, 1, 2) con valor : 61


In [0]:
def coste_inferior(s,costes):
    valor = 0
    for i in range(len(s)):
        valor += costes[i][s[i]]
    for i in range(len(costes)):
        if i not in s:
            valor += min([costes[j][i] for j in range(len(s), len(costes))])
    return valor

In [0]:
def crear_hijos(nodo, n):
    hijos = []
    for i in range(n):
        if i not in nodo:
            hijos.append({'s':nodo+(i,)})
    return hijos

In [0]:
def cota_superior(S,COSTES):
  VALOR = 0
  #Valores establecidos
  for i in range(len(S)):
    VALOR += COSTES[i][S[i]]

  #Estimacion
  for i in range(  len(COSTES)):
    if i not in S:
      VALOR += max( [ COSTES[j][i] for j in range(len(S), len(COSTES))  ])
  return VALOR

In [0]:
def ramificacion_y_poda(COSTES):
#Construccion iterativa de soluciones(arbol). En cada etapa asignamos un agente(ramas).
#Nodos del grafo  { s:(1,2),CI:3,CS:5  }
  #print(COSTES)
  DIMENSION = len(COSTES)  
  MEJOR_SOLUCION=tuple( i for i in range(len(COSTES)) )
  CotaSup = valor(MEJOR_SOLUCION,COSTES)
  #print("Cota Superior:", CotaSup)

  NODOS=[]
  NODOS.append({'s':(), 'ci':coste_inferior((),COSTES)    } )

  iteracion = 0

  while( len(NODOS) > 0):
    iteracion +=1

    nodo_prometedor = [ min(NODOS, key=lambda x:x['ci']) ][0]['s']
    #print("Nodo prometedor:", nodo_prometedor)

    #Ramificacion
    #Se generan los hijos
    HIJOS =[ {'s':x['s'], 'ci':coste_inferior(x['s'], COSTES)   } for x in crear_hijos(nodo_prometedor, DIMENSION) ]

    #Revisamos la cota superior y nos quedamos con la mejor solucion si llegamos a una solucion final
    NODO_FINAL = [x for x in HIJOS if len(x['s']) == DIMENSION  ]
    if len(NODO_FINAL ) >0: 
      #print("\n********Soluciones:",  [x for x in HIJOS if len(x['s']) == DIMENSION  ] )
      if NODO_FINAL[0]['ci'] < CotaSup:
        CotaSup = NODO_FINAL[0]['ci']
        MEJOR_SOLUCION = NODO_FINAL
 
    #Poda
    HIJOS = [x for x in HIJOS if x['ci'] < CotaSup   ]

    #Añadimos los hijos 
    NODOS.extend(HIJOS) 

    #Eliminamos el nodo ramificado
    NODOS =  [  x for x in NODOS if x['s'] != nodo_prometedor    ]
   
    print("La solucion final es:" ,MEJOR_SOLUCION , " en " , iteracion , " iteraciones" , " para dimension: " ,DIMENSION  )
      


ramificacion_y_poda(costes)

La solucion final es: (0, 1, 2, 3)  en  1  iteraciones  para dimension:  4
La solucion final es: (0, 1, 2, 3)  en  2  iteraciones  para dimension:  4
La solucion final es: (0, 1, 2, 3)  en  3  iteraciones  para dimension:  4
La solucion final es: (0, 1, 2, 3)  en  4  iteraciones  para dimension:  4
La solucion final es: (0, 1, 2, 3)  en  5  iteraciones  para dimension:  4
La solucion final es: [{'s': (0, 2, 3, 1), 'ci': 61}]  en  6  iteraciones  para dimension:  4
La solucion final es: [{'s': (0, 2, 3, 1), 'ci': 61}]  en  7  iteraciones  para dimension:  4
La solucion final es: [{'s': (0, 2, 3, 1), 'ci': 61}]  en  8  iteraciones  para dimension:  4
La solucion final es: [{'s': (0, 2, 3, 1), 'ci': 61}]  en  9  iteraciones  para dimension:  4
La solucion final es: [{'s': (0, 2, 3, 1), 'ci': 61}]  en  10  iteraciones  para dimension:  4
La solucion final es: [{'s': (0, 2, 3, 1), 'ci': 61}]  en  11  iteraciones  para dimension:  4
La solucion final es: [{'s': (0, 2, 3, 1), 'ci': 61}]  en  